In [230]:
import os
import nltk
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
from nltk.corpus import stopwords

In [ ]:
project_dir = "C:\\Users\\pyramidheadshark\\Repos\\Steam-AI-Recommendations"
data_path = os.path.join(project_dir, 'data', 'raw', 'Steam Games 2024.csv')
data_path

'C:\\Users\\pyramidheadshark\\Repos\\Steam-AI-Recommendations\\data\\raw\\Steam Games 2024.csv'

In [ ]:
df = pd.read_csv(data_path)

C:\Users\pyramidheadshark\AppData\Local\Temp\ipykernel_5072\2618208116.py:1: DtypeWarning: Columns (0,4,5,6,7,8,17,18,19,20,22,23,24,25,26,27,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [233]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97428 entries, 0 to 97427
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   AppID                       97415 non-null  object
 1   Name                        97412 non-null  object
 2   Release date                97418 non-null  object
 3   Estimated owners            97417 non-null  object
 4   Peak CCU                    97417 non-null  object
 5   Required age                97417 non-null  object
 6   Price                       97416 non-null  object
 7   Discount                    97417 non-null  object
 8   DLC count                   97418 non-null  object
 9   About the game              92537 non-null  object
 10  Supported languages         97410 non-null  object
 11  Full audio languages        97410 non-null  object
 12  Reviews                     10132 non-null  object
 13  Header image                97409 non-null  ob

In [234]:
df.columns

Index(['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU',
       'Required age', 'Price', 'Discount', 'DLC count', 'About the game',
       'Supported languages', 'Full audio languages', 'Reviews',
       'Header image', 'Website', 'Support url', 'Support email', 'Windows',
       'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score',
       'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations',
       'Notes', 'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')

In [235]:
columns_to_drop = ['Discount', 'Reviews', 'Support url', 'Support email', 'Notes']

In [236]:
df.drop(columns=columns_to_drop, inplace=True)

In [237]:
col_dtypes = {
    'AppID': 'Int32',
    'Name': 'object',
    'Release date': 'datetime64[ns]',
    'Estimated owners': 'Int32',
    'Peak CCU': 'Int64',
    'Required age': 'Int8',
    'Price': 'float64',
    'DLC count': 'Int16',
    'About the game': 'object',
    'Supported languages': 'object',
    'Full audio languages': 'object',
    'Header image': 'object',
    'Website': 'object',
    'Windows': 'boolean',
    'Mac': 'boolean',
    'Linux': 'boolean',
    'Metacritic score': 'Int8',
    'Metacritic url': 'object',
    'User score': 'float64',
    'Positive': 'Int32',
    'Negative': 'Int32',
    'Score rank': 'Int8',
    'Achievements': 'Int16',
    'Recommendations': 'Int32',
    'Average playtime forever': 'float64',
    'Average playtime two weeks': 'float64',
    'Median playtime forever': 'float64',
    'Median playtime two weeks': 'float64',
    'Developers': 'object',
    'Publishers': 'object',
    'Categories': 'object',
    'Genres': 'object',
    'Tags': 'object',
    'Screenshots': 'object',
    'Movies': 'object'
}

In [238]:
df

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,CampiÃ£o Games,CampiÃ£o Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosisâ„¢,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSISâ„¢ is a mysterious 2D Platform Puzzler...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.0,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97423,3080940,Femdom Game World: Mom,"Aug 22, 2024",0 - 20000,0,0,4.19,0,. Femdom Game World - is a fascinating series ...,"['English', 'Russian']",...,0,0,0,Femdom Game World,Femdom Game World,"Single-player,Family Sharing",Casual,NaN,https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
97424,2593970,Blocky Farm,"Aug 30, 2024",0 - 0,0,0,8.99,0,Enter the charming world of Blocky Farm where ...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Forever Entertainment S. A.,Forever Entertainment S. A.,"Single-player,Steam Achievements,Partial Contr...","Casual,Simulation,Strategy",NaN,https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
97425,3137150,Infiltrate & Extract,"Aug 30, 2024",0 - 0,0,0,0,0,Mission brief: You are deployed to a heavily g...,['English'],...,0,0,0,Carlos Garrido,DigiPen Intsitute of Technology,Single-player,"Strategy,Free To Play",NaN,https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
97426,3124670,Escape The Garage,"Aug 29, 2024",0 - 0,0,0,4.99,0,"Welcome to Escape The Garage, the thrilling es...",['English'],...,0,0,0,CryneX,"CryneX,CryTechGames","Single-player,Steam Achievements,Family Sharing","Adventure,Casual,Indie",NaN,https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...


In [239]:
df = df[pd.to_numeric(df['AppID'], errors='coerce').notnull()]
df['AppID'] = pd.to_numeric(df['AppID'], errors='coerce').astype('Int32')

C:\Users\pyramidheadshark\AppData\Local\Temp\ipykernel_5072\3577355374.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AppID'] = pd.to_numeric(df['AppID'], errors='coerce').astype('Int32')


In [240]:
for col in ['Windows', 'Mac', 'Linux']:
        df[col] = df[col].astype(str).str.lower() == 'true'

C:\Users\pyramidheadshark\AppData\Local\Temp\ipykernel_5072\2146424897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str).str.lower() == 'true'
C:\Users\pyramidheadshark\AppData\Local\Temp\ipykernel_5072\2146424897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str).str.lower() == 'true'
C:\Users\pyramidheadshark\AppData\Local\Temp\ipykernel_5072\2146424897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [241]:
try:
    df['Release date'] = pd.to_datetime(df['Release date'], infer_datetime_format=True, errors='coerce')
except ValueError as e:
    print(f"Error converting 'Release date' column: {e}.  Check for inconsistencies in date format.")

C:\Users\pyramidheadshark\AppData\Local\Temp\ipykernel_5072\342669661.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Release date'] = pd.to_datetime(df['Release date'], infer_datetime_format=True, errors='coerce')
C:\Users\pyramidheadshark\AppData\Local\Temp\ipykernel_5072\342669661.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Release date'] = pd.to_datetime(df['Release date'], infer_datetime_format=True, errors='coerce')
C:\Users\pyramidheadshark\AppData\Local\Temp\ipykernel_5072\342669661.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [242]:
valid_estimated_owner_ranges = ['0 - 20000', '50000 - 100000', '20000 - 50000', '200000 - 500000', '100000 - 200000',
                    '2000000 - 5000000', '0 - 0', '500000 - 1000000', '1000000 - 2000000',
                    '20000000 - 50000000', '5000000 - 10000000', '10000000 - 20000000',
                    '50000000 - 100000000', '100000000 - 200000000']

In [243]:
df = df[df['Estimated owners'].isin(valid_estimated_owner_ranges)]

In [244]:
try:
    lower_bounds = df['Estimated owners'].str.split(' - ').str[0].astype(float)
    upper_bounds = df['Estimated owners'].str.split(' - ').str[1].astype(float)
    df['Estimated owners'] = (lower_bounds + upper_bounds) / 2
    df['Estimated owners'] = df['Estimated owners'].astype('Int32')
except (ValueError, AttributeError, IndexError) as e:
    print(f"Error processing 'Estimated owners' column (after filtering): {e}")

In [245]:
try:
    df['Peak CCU'] = pd.to_numeric(df['Peak CCU'], errors='coerce').astype('Int32')
except (ValueError, TypeError) as e:
    print(f"Error converting 'Peak CCU' column: {e}.  Attempting more robust cleaning.")
    df['Peak CCU'] = df['Peak CCU'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df['Peak CCU'] = pd.to_numeric(df['Peak CCU'], errors='coerce').astype('Int32')


In [246]:
valid_ages = [0, 17, 18, 10, 13, 16, 6, 14, 12, 15, 3, 21, 7, 5, 20, 1, 19, 9, 11]

In [247]:
try:
    df['Required age'] = pd.to_numeric(df['Required age'], errors='coerce')
    df = df[df['Required age'].isin(valid_ages)]
    df['Required age'] = df['Required age'].astype('Int8')
except (ValueError, TypeError) as e:
    print(f"Error converting 'Required age' column: {e}.  Invalid values found.")


In [248]:
try:
    df['DLC count'] = df['DLC count'].astype('Int16')
except (ValueError, TypeError) as e:
    print(f"Error converting 'DLC count' column: {e}. Cleaning string values.")

    df['DLC count'] = df['DLC count'].astype(str).str.lower()
    df['DLC count'] = df['DLC count'].str.replace(r'[^\d.]', '', regex=True)
    df['DLC count'] = df['DLC count'].replace(['n/a', 'none', ''], np.nan)

    df['DLC count'] = pd.to_numeric(df['DLC count'], errors='coerce').astype('Int16')


Error converting 'DLC count' column: cannot safely cast non-equivalent object to int16. Cleaning string values.


In [249]:
try:
    df['Metacritic score'] = df['Metacritic score'].replace('<NA>', np.nan)
    df['Metacritic score'] = pd.to_numeric(df['Metacritic score'], errors='coerce').astype('Int8')

except (ValueError, TypeError) as e:
    print(f"Error converting 'Metacritic score' column: {e}.  Check for unexpected values.")

In [250]:
try:
    df['Positive'] = df['Positive'].replace('<NA>', np.nan)
    df['Positive'] = df['Positive'].round().astype('Int32')

except (ValueError, TypeError) as e:
    print(f"Error converting 'Positive' column: {e}. Cleaning non-numeric values.")
    df['Positive'] = df['Positive'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df['Positive'] = pd.to_numeric(df['Positive'], errors='coerce').astype('Int32')

Error converting 'Positive' column: cannot safely cast non-equivalent object to int32. Cleaning non-numeric values.


In [251]:
try:
    df['Negative'] = df['Negative'].replace('<NA>', np.nan)
    df['Negative'] = df['Negative'].round().astype('Int32')

except (ValueError, TypeError) as e:
    print(f"Error converting 'Negative' column: {e}. Cleaning non-numeric values.")
    df['Negative'] = df['Negative'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df['Negative'] = pd.to_numeric(df['Negative'], errors='coerce').astype('Int32')

Error converting 'Negative' column: cannot safely cast non-equivalent object to int32. Cleaning non-numeric values.


In [252]:
try:
    df['Score rank'] = df['Score rank'].replace('<NA>', np.nan)
    df['Score rank'] = df['Positive'].round().astype('Int8')

except (ValueError, TypeError) as e:
    print(f"Error converting 'Score rank' column: {e}. Cleaning non-numeric values.")

    df['Score rank'] = df['Score rank'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df['Score rank'] = df['Score rank'].astype('Int8')

In [253]:
df['Achievements'] = df['Achievements'].replace(['<NA>', ''], np.nan)
df['Achievements'] = pd.to_numeric(df['Achievements'], errors='coerce').astype('Int16')

In [254]:
try:
    df['Recommendations'] = df['Recommendations'].replace('<NA>', np.nan)
    df['Recommendations'] = df['Recommendations'].round().astype('Int32')

except (ValueError, TypeError) as e:
    print(f"Error converting 'Recommendations' column: {e}. Cleaning non-numeric values.")
    df['Recommendations'] = df['Recommendations'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df['Recommendations'] = pd.to_numeric(df['Recommendations'], errors='coerce').astype('Int32')

Error converting 'Recommendations' column: cannot safely cast non-equivalent object to int32. Cleaning non-numeric values.


In [255]:
for col, dtype in col_dtypes.items():
    if col.lower() not in [
        'appid', 'windows', 'mac', 'linux',
        'release date', 'estimated owners', 'required age',
        'dlc count', 'metacritic score', 'positive',
        'negative', 'score rank', 'achievements',
        'recommendations'
        ]:
        try:
            df[col] = df[col].astype(dtype, errors='raise')
        except (ValueError, TypeError) as e:
            print(f"Error converting column '{col}' to type '{dtype}': {e}")
            if dtype in ['Int64', 'int64']:
                df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
            elif dtype == 'float64':
                df[col] = pd.to_numeric(df[col], errors='coerce')
            else:
                print(f"Fallback to object dtype. Column {col} not converted.")

In [256]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97410 entries, 0 to 97427
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   AppID                       97410 non-null  Int32         
 1   Name                        97404 non-null  object        
 2   Release date                97279 non-null  datetime64[ns]
 3   Estimated owners            97410 non-null  Int32         
 4   Peak CCU                    97410 non-null  Int64         
 5   Required age                97410 non-null  Int8          
 6   Price                       97410 non-null  float64       
 7   DLC count                   97410 non-null  Int16         
 8   About the game              92529 non-null  object        
 9   Supported languages         97402 non-null  object        
 10  Full audio languages        97402 non-null  object        
 11  Header image                97402 non-null  object        


In [262]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97410 entries, 0 to 97427
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   AppID                       97410 non-null  Int32         
 1   Name                        97404 non-null  object        
 2   Release date                97279 non-null  datetime64[ns]
 3   Estimated owners            97410 non-null  Int32         
 4   Peak CCU                    97410 non-null  Int64         
 5   Required age                97410 non-null  Int8          
 6   Price                       97410 non-null  float64       
 7   DLC count                   97410 non-null  Int16         
 8   About the game              92529 non-null  object        
 9   Supported languages         97402 non-null  object        
 10  Full audio languages        97402 non-null  object        
 11  Header image                97402 non-null  object        


In [263]:
df

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,2008-10-21,10000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0.0,0.0,0.0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,2017-10-12,10000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0.0,0.0,0.0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,2021-11-17,10000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0.0,0.0,0.0,CampiÃ£o Games,CampiÃ£o Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosisâ„¢,2020-07-23,10000,0,0,5.99,0,HENOSISâ„¢ is a mysterious 2D Platform Puzzler...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0.0,0.0,0.0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,2020-02-03,10000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0.0,0.0,0.0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97423,3080940,Femdom Game World: Mom,2024-08-22,10000,0,0,4.19,0,. Femdom Game World - is a fascinating series ...,"['English', 'Russian']",...,0.0,0.0,0.0,Femdom Game World,Femdom Game World,"Single-player,Family Sharing",Casual,NaN,https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
97424,2593970,Blocky Farm,2024-08-30,0,0,0,8.99,0,Enter the charming world of Blocky Farm where ...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0.0,0.0,0.0,Forever Entertainment S. A.,Forever Entertainment S. A.,"Single-player,Steam Achievements,Partial Contr...","Casual,Simulation,Strategy",NaN,https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
97425,3137150,Infiltrate & Extract,2024-08-30,0,0,0,0.00,0,Mission brief: You are deployed to a heavily g...,['English'],...,0.0,0.0,0.0,Carlos Garrido,DigiPen Intsitute of Technology,Single-player,"Strategy,Free To Play",NaN,https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
97426,3124670,Escape The Garage,2024-08-29,0,0,0,4.99,0,"Welcome to Escape The Garage, the thrilling es...",['English'],...,0.0,0.0,0.0,CryneX,"CryneX,CryTechGames","Single-player,Steam Achievements,Family Sharing","Adventure,Casual,Indie",NaN,https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...


In [264]:
for col in df.columns:
    if df[col].dtype == object:
        df[col] = df[col].astype(str).str.replace(r'[^\x00-\x7F]+', '', regex=True)

In [265]:
duplicate_rows = df[df.duplicated(subset=['Name'], keep=False)]
duplicate_rows

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
59,1135380,,2019-10-03,10000,0,0,3.99,0,"With the rise of professional soldiers, the st...",['Simplified Chinese'],...,0.0,0.0,0.0,,,Single-player,Indie,"Sexual Content,Nudity,Indie,Visual Novel",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
82,1023420,,2020-07-31,10000,0,0,24.99,0,_________________________________ ! ! ...,['Japanese'],...,0.0,0.0,0.0,Flying Panjandrum /,Flying Panjandrum /,Single-player,"Adventure,Indie,Simulation,Strategy","Adventure,Indie,Simulation,Strategy,Anime,Turn...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
105,1943590,Playtest,2022-03-24,0,0,0,0.00,0,nan,[],...,0.0,0.0,0.0,nan,nan,nan,nan,nan,nan,nan
111,587490,Dark Throne,2017-03-03,10000,0,0,0.99,0,Dark Throne is a game in which you play as a k...,['English'],...,0.0,0.0,0.0,Marcin Mikoajczyk,Back To Basics Gaming,"Single-player,Steam Achievements,Partial Contr...","Action,Adventure,Indie,RPG","Action,Adventure,Indie,RPG,Pixel Graphics",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
148,717980,,2018-05-11,10000,0,0,6.99,0,": ,.''.''.,,.'',,,? {STEAM_APP_IMAGE}/extras/t...",['Simplified Chinese'],...,0.0,3.0,0.0,,,"Single-player,Steam Achievements","Adventure,Indie,RPG","Adventure,Indie,RPG,Drama,Turn-Based,Story Rich",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97286,3134030,The Last Hero,2024-08-25,10000,0,0,2.99,0,Welcome to Corli! you are Hugh Ashdown a sword...,['English'],...,0.0,0.0,0.0,Percy Rogers-Llanes,Nostalgic Games,"Single-player,Full controller support,Family S...","Action,Adventure,Casual,Indie",nan,https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
97324,3036480,,2024-08-21,0,0,0,0.00,0,Entries for the Champion of Hanny Crushing are...,['Japanese'],...,0.0,0.0,0.0,"Alicesoft,Orgesta Inc.",Alicesoft,"Single-player,Full controller support","Action,Free To Play",nan,https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
97376,3109990,,2024-08-23,10000,0,0,2.79,0,Saving Diary 'is an independent single player ...,['Simplified Chinese'],...,0.0,0.0,0.0,,,"Single-player,Family Sharing","RPG,Strategy","Strategy,Card Game,Card Battler,Turn-Based Tac...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
97405,3167240,Playtest,2024-08-21,0,0,0,0.00,0,nan,[],...,0.0,0.0,0.0,nan,nan,nan,nan,nan,nan,nan


In [266]:
df_sorted = df.sort_values(['Estimated owners', 'Release date'], ascending=[False, False])
df_deduped = df_sorted.drop_duplicates(subset=['Name'], keep='first')
df_deduped

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
17585,570,Dota 2,2013-07-09,150000000,558759,0,0.00,2,"The most-played game on Steam. Every day, mill...","['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...",...,1647.0,880.0,749.0,Valve,Valve,"Multi-player,Co-op,Steam Trading Cards,Steam W...","Action,Free to Play,Strategy","Free to Play,MOBA,Multiplayer,Strategy,e-sport...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
96966,2358720,Black Myth: Wukong,2024-08-19,75000000,1214621,0,59.99,1,Black Myth: Wukong is an action RPG rooted in ...,"['English', 'French', 'German', 'Spanish - Spa...",...,1855.0,1507.0,1507.0,Game Science,Game Science,"Single-player,Steam Achievements,Full controll...","Action,Adventure,RPG","Mythology,Action RPG,Action,RPG,Souls-like,Com...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
7030,1063730,New World,2021-09-28,75000000,16688,0,39.99,0,"Explore a thrilling, open-world MMO filled wit...","['English', 'French', 'Italian', 'German', 'Sp...",...,607.0,3093.0,259.0,Amazon Games,Amazon Games,"Multi-player,MMO,PvP,Online PvP,Co-op,Online C...","Action,Adventure,Massively Multiplayer,RPG","Massively Multiplayer,Open World,MMORPG,RPG,Ad...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
8885,578080,PUBG: BATTLEGROUNDS,2017-12-21,75000000,275374,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","['English', 'Korean', 'Simplified Chinese', 'F...",...,790.0,7134.0,152.0,"KRAFTON, Inc.","KRAFTON, Inc.","Multi-player,PvP,Online PvP,Stats,Remote Play ...","Action,Adventure,Free to Play,Massively Multip...","Survival,Shooter,Multiplayer,Battle Royale,FPS...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
46158,730,Counter-Strike: Global Offensive,2012-08-21,75000000,825215,0,0.00,1,Counter-Strike: Global Offensive (CS: GO) expa...,"['Czech', 'Danish', 'Dutch', 'English', 'Finni...",...,777.0,6357.0,253.0,"Valve,Hidden Path Entertainment",Valve,"Multi-player,Steam Achievements,Full controlle...","Action,Free to Play","FPS,Shooter,Multiplayer,Competitive,Action,Tea...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/8...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25774,1670240,Echo of Ayllu,NaT,0,0,0,0.00,0,The project is currently in early access. Offi...,"['English', 'French', 'Spanish - Spain']",...,0.0,0.0,0.0,Gaiac Games,Gaiac Games,"Single-player,Multi-player,PvP,Online PvP,Stea...","Strategy,Early Access",nan,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
36491,1959960,Burg,NaT,0,0,0,0.00,0,Burg is a First-Person Multiplayer Hide and Se...,['English'],...,0.0,0.0,0.0,Barji,Barji,"Multi-player,PvP,Online PvP","Action,Casual,Free to Play,Indie",nan,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
58456,1960630,Cursed 2,NaT,0,0,0,0.00,0,Inspired by classic point and click horror gam...,['English'],...,0.0,0.0,0.0,Disaster Squad Productions,Disaster Squad Productions,Single-player,"Action,Adventure,Free to Play,Indie",nan,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
63418,2131220,Neisis,NaT,0,0,0,0.00,0,Neisis is an immersive medieval and magic MMOR...,['English'],...,0.0,0.0,0.0,Antoine J.,Antoine J.,"Multi-player,MMO,PvP,Online PvP,Co-op,Online C...","Action,Adventure,Massively Multiplayer,RPG",nan,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [267]:
df = df_deduped

In [268]:
mask = ~df['Name'].str.isalnum() & ~df['Name'].str.isalpha()
df_filtered = df[mask]
df_filtered

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
17585,570,Dota 2,2013-07-09,150000000,558759,0,0.00,2,"The most-played game on Steam. Every day, mill...","['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...",...,1647.0,880.0,749.0,Valve,Valve,"Multi-player,Co-op,Steam Trading Cards,Steam W...","Action,Free to Play,Strategy","Free to Play,MOBA,Multiplayer,Strategy,e-sport...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
96966,2358720,Black Myth: Wukong,2024-08-19,75000000,1214621,0,59.99,1,Black Myth: Wukong is an action RPG rooted in ...,"['English', 'French', 'German', 'Spanish - Spa...",...,1855.0,1507.0,1507.0,Game Science,Game Science,"Single-player,Steam Achievements,Full controll...","Action,Adventure,RPG","Mythology,Action RPG,Action,RPG,Souls-like,Com...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
7030,1063730,New World,2021-09-28,75000000,16688,0,39.99,0,"Explore a thrilling, open-world MMO filled wit...","['English', 'French', 'Italian', 'German', 'Sp...",...,607.0,3093.0,259.0,Amazon Games,Amazon Games,"Multi-player,MMO,PvP,Online PvP,Co-op,Online C...","Action,Adventure,Massively Multiplayer,RPG","Massively Multiplayer,Open World,MMORPG,RPG,Ad...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
8885,578080,PUBG: BATTLEGROUNDS,2017-12-21,75000000,275374,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","['English', 'Korean', 'Simplified Chinese', 'F...",...,790.0,7134.0,152.0,"KRAFTON, Inc.","KRAFTON, Inc.","Multi-player,PvP,Online PvP,Stats,Remote Play ...","Action,Adventure,Free to Play,Massively Multip...","Survival,Shooter,Multiplayer,Battle Royale,FPS...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
46158,730,Counter-Strike: Global Offensive,2012-08-21,75000000,825215,0,0.00,1,Counter-Strike: Global Offensive (CS: GO) expa...,"['Czech', 'Danish', 'Dutch', 'English', 'Finni...",...,777.0,6357.0,253.0,"Valve,Hidden Path Entertainment",Valve,"Multi-player,Steam Achievements,Full controlle...","Action,Free to Play","FPS,Shooter,Multiplayer,Competitive,Action,Tea...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/8...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4542,855060,Huawei VR 2,NaT,0,0,0,0.00,0,Huawei VR 2 is the first product which can acc...,[],...,0.0,0.0,0.0,nan,nan,nan,nan,nan,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
16289,1810290,HUAWEI VR Glass 6DoF Driver,NaT,0,0,0,0.00,0,The VR 6DoF Motion Tracking kit driver is comp...,"['Simplified Chinese', 'English']",...,0.0,0.0,0.0,"Huawei Device (Dongguan) Co., Ltd","Huawei Device (Dongguan) Co., Ltd",Full controller support,"Software Training,Utilities",nan,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
20890,877280,SAO Utils 2: Progressive,NaT,0,0,0,0.00,0,SAO Utils 2: Progressive provides you a fresh-...,"['English', 'Japanese', 'Simplified Chinese', ...",...,0.0,0.0,0.0,Joshua Chen,Studio GPBeta,Steam Workshop,"Utilities,Early Access",nan,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
25774,1670240,Echo of Ayllu,NaT,0,0,0,0.00,0,The project is currently in early access. Offi...,"['English', 'French', 'Spanish - Spain']",...,0.0,0.0,0.0,Gaiac Games,Gaiac Games,"Single-player,Multi-player,PvP,Online PvP,Stea...","Strategy,Early Access",nan,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [269]:
df = df_filtered

In [270]:
df[[
    'Metacritic score', 'User score', 'Positive',
    'Negative', 'Score rank', 'Achievements', 'Recommendations'
    ]] = df[[
        'Metacritic score', 'User score', 'Positive',
        'Negative', 'Score rank', 'Achievements', 'Recommendations'
    ]].fillna(0)

C:\Users\pyramidheadshark\AppData\Local\Temp\ipykernel_5072\4004226968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[[


In [271]:
df[[
    'Average playtime forever', 'Average playtime two weeks',
    'Median playtime forever', 'Median playtime two weeks'
    ]] = df[[
        'Average playtime forever', 'Average playtime two weeks',
        'Median playtime forever', 'Median playtime two weeks'
    ]].fillna(0.0)

C:\Users\pyramidheadshark\AppData\Local\Temp\ipykernel_5072\3982059331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[[


In [272]:
mask = df['Release date'].notna()
df_filtered = df[mask]

In [273]:
df = df_filtered

In [274]:
df_final = df
df_final

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
17585,570,Dota 2,2013-07-09,150000000,558759,0,0.00,2,"The most-played game on Steam. Every day, mill...","['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...",...,1647.0,880.0,749.0,Valve,Valve,"Multi-player,Co-op,Steam Trading Cards,Steam W...","Action,Free to Play,Strategy","Free to Play,MOBA,Multiplayer,Strategy,e-sport...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
96966,2358720,Black Myth: Wukong,2024-08-19,75000000,1214621,0,59.99,1,Black Myth: Wukong is an action RPG rooted in ...,"['English', 'French', 'German', 'Spanish - Spa...",...,1855.0,1507.0,1507.0,Game Science,Game Science,"Single-player,Steam Achievements,Full controll...","Action,Adventure,RPG","Mythology,Action RPG,Action,RPG,Souls-like,Com...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
7030,1063730,New World,2021-09-28,75000000,16688,0,39.99,0,"Explore a thrilling, open-world MMO filled wit...","['English', 'French', 'Italian', 'German', 'Sp...",...,607.0,3093.0,259.0,Amazon Games,Amazon Games,"Multi-player,MMO,PvP,Online PvP,Co-op,Online C...","Action,Adventure,Massively Multiplayer,RPG","Massively Multiplayer,Open World,MMORPG,RPG,Ad...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
8885,578080,PUBG: BATTLEGROUNDS,2017-12-21,75000000,275374,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","['English', 'Korean', 'Simplified Chinese', 'F...",...,790.0,7134.0,152.0,"KRAFTON, Inc.","KRAFTON, Inc.","Multi-player,PvP,Online PvP,Stats,Remote Play ...","Action,Adventure,Free to Play,Massively Multip...","Survival,Shooter,Multiplayer,Battle Royale,FPS...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
46158,730,Counter-Strike: Global Offensive,2012-08-21,75000000,825215,0,0.00,1,Counter-Strike: Global Offensive (CS: GO) expa...,"['Czech', 'Danish', 'Dutch', 'English', 'Finni...",...,777.0,6357.0,253.0,"Valve,Hidden Path Entertainment",Valve,"Multi-player,Steam Achievements,Full controlle...","Action,Free to Play","FPS,Shooter,Multiplayer,Competitive,Action,Tea...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/8...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34931,211209,Wonder Boy in Monster World,2012-05-02,0,0,0,0.99,0,"Monsters have taken over the land, and every d...",['English'],...,0.0,0.0,0.0,nan,SEGA,Single-player,nan,nan,https://cdn.akamai.steamstatic.com/steam/apps/...,nan
58771,90500,Guardians of Graxia,2010-11-01,0,0,0,0.00,0,Guardians of Graxia takes on the form of a car...,['English'],...,0.0,0.0,0.0,nan,nan,"Single-player,Steam Achievements",Strategy,nan,https://cdn.akamai.steamstatic.com/steam/apps/...,nan
46208,33730,18 Wheels of Steel: Extreme Trucker,2009-09-23,0,0,0,9.99,0,You da Boss! Move it better and faster while s...,['English'],...,0.0,0.0,0.0,nan,Cosmi Valusoft,Single-player,Simulation,nan,https://cdn.akamai.steamstatic.com/steam/apps/...,nan
31737,11390,Crash Time 2,2009-08-27,0,0,0,1.24,0,Solve exciting criminal cases on the mean stre...,"['English', 'French', 'German']",...,0.0,0.0,0.0,nan,Meridian4,"Single-player,Remote Play Together","Action,Racing,Simulation",nan,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [275]:
processed_path = os.path.join(project_dir, 'data', 'processed', 'Steam Games 2024.csv')

In [276]:
df_final.to_csv(processed_path, index=False)

In [277]:
selected_columns = ['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU',
                    'Required age', 'Price', 'DLC count', 'Windows', 'Mac', 'Linux',
                    'Metacritic score', 'User score', 'Positive', 'Negative', 'Score rank',
                    'Achievements', 'Recommendations', 'Average playtime forever',
                    'Average playtime two weeks', 'Median playtime forever',
                    'Median playtime two weeks', 'Categories', 'Genres', 'Tags']

In [278]:
df_selected = df[selected_columns]

nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))

def clean_text(text):
    if pd.isnull(text):
        return ""
    words = text.lower().split()
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

def preprocess_dataframe(df):
    text_columns = ['Name', 'Categories', 'Genres', 'Tags']
    for col in text_columns:
        if df[col].dtype == 'object':
            df[col] = df[col].apply(clean_text)
    return df

df_cleaned = preprocess_dataframe(df_selected)

C:\Users\pyramidheadshark\AppData\Local\Temp\ipykernel_5072\954663384.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(clean_text)


In [279]:
profile = ProfileReport(df_cleaned, title="Profile Report")

In [280]:
profile_path = os.path.join(project_dir, 'src', 'visualization', 'Report.html')

In [281]:
profile.to_file(profile_path)

c:\Users\pyramidheadshark\.conda\envs\Steam-AI-Recommendations\Lib\site-packages\ydata_profiling\utils\dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]